In [107]:
'''
Se usara POO, con nodos y aristas
Se va a considerar que los nodos tienen un 
- nombre
- descripción
- tiempo optimista
- tiempo pesimista
- tiempo más probable
- lista de nodos que apuntan a él (precedentes)

Se calculará
- tiempo pert
- camino crítico
- tiempo early
- tiempo late
'''
import datetime

class PERTNode:
  '''
  clase para el nodo PERT
  '''
  def __init__(self, name):
    '''
    constructor
    '''
    self.name = name
    self.t_e = 0
    self.t_l = 0
    self.h_nodo = 0
    self.fecha = None
    
    # edges que llegan
    self.inc = []
    # edges que salen
    self.out = []
    
  def addOutEdge(self, edge):
    '''
    método para agregar una edge saliente
    '''
    self.out.append(edge)
  
  
  def addIncEdge(self, edge):
    '''
    método para agregar una edge entrante
    '''
    self.inc.append(edge)
  
  def fTEarly(self, is_first = False):
    '''
    método para calcular el tiempo early
    '''
    if is_first:
      self.t_e = 0
      return
    
    # para cada arista que llega al nodo:
    #   tomamos el tiempo early del nodo del cual sale la arista +
    #   el tiempo PERT de la arista
    self.t_e = max([ed.n_sal.t_e + ed.t_pert for ed in self.inc])
  
  
  def fTLate(self, is_last = False):
    '''
    método para calcular el tiempo late
    '''
    if is_last:
      self.t_l = self.t_e
      return
    
    # para cada arista que sale del nodo:
    #   tomamos el tiempo late del nodo al cual entra la arista +
    #   el tiempo PERT de la arista
    self.t_l = min([ed.n_ent.t_l - ed.t_pert for ed in self.out])
  
  def fHolgura(self):
    '''
    método para calcular la holgura de un suceso
    '''
    self.h_nodo = self.t_l - self.t_e
    
  def printNode(self):
    '''
    método para imprimir información del nodin
    '''
    print("\n------ NODO %d -----"%(self.name))
    print("tiempo early:\t", self.t_e)
    print("tiempo late: \t", self.t_l)
    print("fecha:       \t", self.fecha)
    

class PERTEdge:
  '''
  clase para la arista PERT
  '''
  
  def __init__(self, name, to, tp, tn, n_sal, n_ent, prec={}, desc = None):
    '''
    constructor de los nodos pert
    
    n_sal -> nodo del cual sale
    n_ent -> nodo al que entra
    prec -> conjunto con nombres de instancias de nodos pert definidos previamente
    '''
    self.name = name
    self.desc = desc
    self.n_sal = n_sal
    self.n_ent = n_ent
    self.to = to
    self.tp = tp
    self.tn = tn
    self.prec = prec
    self.t_holg = None
    
    # tiempo PERT
    self.t_pert = (to + 4*tn + tp) / 6
    
    self.t_early = - float("inf")
    self.t_late = float("inf")
  
  def fHolguraE(self):
    '''
    método para calcular la holgura de la arista
    '''
    self.t_holg = self.n_ent.t_l - self.n_sal.t_e - self.t_pert
    print(self.n_ent.t_l - self.n_sal.t_e - self.t_pert)
    print(self.t_holg)
    #self.t_holg =  self.n_sal.t_e - self.n_ent.t_l
    
  def printEdge(self):
    '''
    método para imprimir una arista
    '''
    print("\n--- ARISTA: \t", self.name)
    print("--- ORIGEN: \t", self.n_sal.name)
    print("--- DESTINO:\t", self.n_ent.name)
    print("--- Ti PERT:\t", self.t_pert)
    print("--- HOLGURA:\t", self.t_holg)
    print("--- t_e sal:\t", self.n_sal.t_e)
    print("--- t_l ent:\t", self.n_ent.t_l)
    print("--- HOLG  2:\t", self.n_ent.t_l - self.n_sal.t_e - self.t_pert)
  
  def printCritialEdge(self):
    if abs(self.n_ent.t_l - self.n_sal.t_e - self.t_pert) < 0.5:
      print("\n--- ARISTA: \t", self.name)
      print("--- ORIGEN: \t", self.n_sal.name)
      print("--- DESTINO:\t", self.n_ent.name)
      print("--- Ti PERT:\t", self.t_pert)
      #print("--- HOLGURA:\t", self.t_holg)
     #print("--- t_e sal:\t", self.n_sal.t_e)
     #print("--- t_l ent:\t", self.n_ent.t_l)
      print("--- HOLG  2:\t", self.n_ent.t_l - self.n_sal.t_e - self.t_pert)    
    
class PERTGraph:
  '''
  clase para el grafo PERT
  '''
  def __init__(self):
    '''
    constructor
    '''
    
    # lista de nodos
    self.nodes = []
    
    # lista de adyacencia, con llaves las instancias
    self.edges = {}
    
    # tamaño
    self.sz = 0
  
  def add_node(self, p_node):
    '''
    método para agregar un nodo al grafo PERT
    '''
    self.sz += 1
    # agregamos un nodin
    self.nodes.append(p_node)
    
  def add_edge(self, p_edge):
    '''
    método para agregar esges al grafo pert
    '''
    
    sal = p_edge.n_sal
    ent = p_edge.n_ent
    
    sal.addOutEdge(p_edge)
    ent.addIncEdge(p_edge)
      
      
  def calculateTEarly(self):
    '''
    método para calcular el tiempo early de todos los elementos
    '''
    for i in range(self.sz):
      # condiciones sobre los tiempos early y late iniciales
      if i == 0:
        is_first = True
      else:
        is_first = False
      
      self.nodes[i].fTEarly(is_first)
  
      
  
  def calculateTLate(self):
    '''
    método para calcular el tiempo late de todos los elementos
    '''
    for i in range(self.sz):
      # condiciones sobre los tiempos early y late iniciales
      k = self.sz - i -1
      if k == self.sz -1:
        is_last = True
      else:
        is_last = False
      
      self.nodes[k].fTLate(is_last)
        
        
  def fAllHolguras(self):
    '''
    método para calcular las holguras de las aristas y los nodos
    '''
    for i in range(self.sz):
      # holgura de los nodos
      self.nodes[i].fHolgura()
    
      # holgura de cada arista que sale de ese nodo
      for ed in self.nodes[i].out:
        #
        ed.fHolguraE()
        print(ed.t_holg)
        print()
    
    
  def printAllNodes(self):
    '''
    método para imprimir todos los nodines
    '''
    for nodito in self.nodes:
      nodito.printNode()
  
  def printCriticalPath(self):
    '''
    método para imprimir el camino crítico
    '''
    for nodito in self.nodes:
      for ed in nodito.out:
        ed.printCritialEdge()
        if abs(ed.t_holg) < 0.5:
          ed.printEdge()
  
  #def importCSV(self, path2csv):
   # '''
    #método para importar info de un CSV
    #'''
    

In [108]:
# tenemos 29 noditos, los vamos a anexar al grafo pert
grafoP = PERTGraph()
for i in range(29):
  grafoP.add_node(PERTNode(i))


In [109]:
# agregamos todas las edges
grafoP.add_edge(PERTEdge("A1", 2,4,3, grafoP.nodes[0], grafoP.nodes[1]))

grafoP.add_edge(PERTEdge("A2", 2,4,3, grafoP.nodes[1], grafoP.nodes[2]))

grafoP.add_edge(PERTEdge("A3", 1,2,1.5, grafoP.nodes[2], grafoP.nodes[3]))

grafoP.add_edge(PERTEdge("A4", 1,3,2, grafoP.nodes[3], grafoP.nodes[4]))

grafoP.add_edge(PERTEdge("A5", 3,5,4, grafoP.nodes[1], grafoP.nodes[5]))
grafoP.add_edge(PERTEdge("A5", 3,5,4, grafoP.nodes[4], grafoP.nodes[5]))

grafoP.add_edge(PERTEdge("A6", 3,5,4, grafoP.nodes[1], grafoP.nodes[6]))
grafoP.add_edge(PERTEdge("A6", 3,5,4, grafoP.nodes[3], grafoP.nodes[6]))

grafoP.add_edge(PERTEdge("A7", 2,4,3, grafoP.nodes[5], grafoP.nodes[7]))

grafoP.add_edge(PERTEdge("A8", 2,4,3, grafoP.nodes[6], grafoP.nodes[8]))

grafoP.add_edge(PERTEdge("A9", 1,3,2, grafoP.nodes[7], grafoP.nodes[9]))
grafoP.add_edge(PERTEdge("A9", 1,3,2, grafoP.nodes[8], grafoP.nodes[9]))

grafoP.add_edge(PERTEdge("A10",1,2,1.5, grafoP.nodes[1], grafoP.nodes[10]))
grafoP.add_edge(PERTEdge("A10",1,2,1.5, grafoP.nodes[2], grafoP.nodes[10]))

grafoP.add_edge(PERTEdge("A11",1,3,2, grafoP.nodes[10], grafoP.nodes[11]))

grafoP.add_edge(PERTEdge("A12",1,2,1.5, grafoP.nodes[10], grafoP.nodes[12]))

grafoP.add_edge(PERTEdge("A13",2,3,2.5, grafoP.nodes[10], grafoP.nodes[13]))

grafoP.add_edge(PERTEdge("A14",3,5,4, grafoP.nodes[11], grafoP.nodes[14]))
grafoP.add_edge(PERTEdge("A14",3,5,4, grafoP.nodes[12], grafoP.nodes[14]))
grafoP.add_edge(PERTEdge("A14",3,5,4, grafoP.nodes[13], grafoP.nodes[14]))

grafoP.add_edge(PERTEdge("A15",3,5,4, grafoP.nodes[11], grafoP.nodes[15]))
grafoP.add_edge(PERTEdge("A15",3,5,4, grafoP.nodes[12], grafoP.nodes[15]))
grafoP.add_edge(PERTEdge("A15",3,5,4, grafoP.nodes[13], grafoP.nodes[15]))

grafoP.add_edge(PERTEdge("A16",2,4,3, grafoP.nodes[13], grafoP.nodes[16]))
grafoP.add_edge(PERTEdge("A16",2,4,3, grafoP.nodes[14], grafoP.nodes[16]))

grafoP.add_edge(PERTEdge("A17",2,4,3, grafoP.nodes[13], grafoP.nodes[17]))
grafoP.add_edge(PERTEdge("A17",2,4,3, grafoP.nodes[15], grafoP.nodes[17]))

grafoP.add_edge(PERTEdge("A18",3,4,5, grafoP.nodes[16], grafoP.nodes[18]))
grafoP.add_edge(PERTEdge("A18",3,4,5, grafoP.nodes[17], grafoP.nodes[18]))

grafoP.add_edge(PERTEdge("A19",1,2,1.5, grafoP.nodes[1], grafoP.nodes[19]))
grafoP.add_edge(PERTEdge("A19",1,2,1.5, grafoP.nodes[2], grafoP.nodes[19]))

grafoP.add_edge(PERTEdge("A20",2,3,2.5, grafoP.nodes[19], grafoP.nodes[20]))

grafoP.add_edge(PERTEdge("A21",2,3,2.5, grafoP.nodes[14], grafoP.nodes[21]))
grafoP.add_edge(PERTEdge("A21",2,3,2.5, grafoP.nodes[15], grafoP.nodes[21]))
grafoP.add_edge(PERTEdge("A21",2,3,2.5, grafoP.nodes[19], grafoP.nodes[21]))

grafoP.add_edge(PERTEdge("A22",1,2,1.5, grafoP.nodes[7], grafoP.nodes[22]))
grafoP.add_edge(PERTEdge("A22",1,2,1.5, grafoP.nodes[8], grafoP.nodes[22]))
grafoP.add_edge(PERTEdge("A22",1,2,1.5, grafoP.nodes[19], grafoP.nodes[22]))

grafoP.add_edge(PERTEdge("A23",2,3,2.5, grafoP.nodes[20], grafoP.nodes[23]))
grafoP.add_edge(PERTEdge("A23",2,3,2.5, grafoP.nodes[21], grafoP.nodes[23]))
grafoP.add_edge(PERTEdge("A23",2,3,2.5, grafoP.nodes[22], grafoP.nodes[23]))

grafoP.add_edge(PERTEdge("A24",1,2,1.5, grafoP.nodes[9], grafoP.nodes[24]))
grafoP.add_edge(PERTEdge("A24",1,2,1.5, grafoP.nodes[18], grafoP.nodes[24]))
grafoP.add_edge(PERTEdge("A24",1,2,1.5, grafoP.nodes[23], grafoP.nodes[24]))

grafoP.add_edge(PERTEdge("A25",2,4,3, grafoP.nodes[24], grafoP.nodes[25]))

grafoP.add_edge(PERTEdge("A26",2,5,4, grafoP.nodes[25], grafoP.nodes[26]))

grafoP.add_edge(PERTEdge("A27",1,3,2, grafoP.nodes[26], grafoP.nodes[27]))

grafoP.add_edge(PERTEdge("A28",3,5,4, grafoP.nodes[27], grafoP.nodes[28]))

In [110]:
# calculamos las holguras
grafoP.fAllHolguras()

-3.0
-3.0
-3.0

-3.0
-3.0
-3.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-1.5
-1.5
-1.5

-1.5
-1.5
-1.5

-1.5
-1.5
-1.5

-1.5
-1.5
-1.5

-1.5
-1.5
-1.5

-2.0
-2.0
-2.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-3.0
-3.0
-3.0

-3.0
-3.0
-3.0

-2.0
-2.0
-2.0

-1.5
-1.5
-1.5

-2.0
-2.0
-2.0

-1.5
-1.5
-1.5

-1.5
-1.5
-1.5

-2.0
-2.0
-2.0

-1.5
-1.5
-1.5

-2.5
-2.5
-2.5

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-4.0
-4.0
-4.0

-3.0
-3.0
-3.0

-3.0
-3.0
-3.0

-3.0
-3.0
-3.0

-2.5
-2.5
-2.5

-3.0
-3.0
-3.0

-2.5
-2.5
-2.5

-4.5
-4.5
-4.5

-4.5
-4.5
-4.5

-1.5
-1.5
-1.5

-2.5
-2.5
-2.5

-2.5
-2.5
-2.5

-1.5
-1.5
-1.5

-2.5
-2.5
-2.5

-2.5
-2.5
-2.5

-2.5
-2.5
-2.5

-1.5
-1.5
-1.5

-3.0
-3.0
-3.0

-3.8333333333333335
-3.8333333333333335
-3.8333333333333335

-2.0
-2.0
-2.0

-4.0
-4.0
-4.0



In [111]:
# calculamos tiempos early y late e imprimimos los nodos
grafoP.calculateTEarly()                                   
grafoP.calculateTLate()                                    
grafoP.printAllNodes()     


------ NODO 0 -----
tiempo early:	 0
tiempo late: 	 -3.552713678800501e-15
fecha:       	 None

------ NODO 1 -----
tiempo early:	 3.0
tiempo late: 	 2.9999999999999964
fecha:       	 None

------ NODO 2 -----
tiempo early:	 6.0
tiempo late: 	 5.9999999999999964
fecha:       	 None

------ NODO 3 -----
tiempo early:	 7.5
tiempo late: 	 8.499999999999996
fecha:       	 None

------ NODO 4 -----
tiempo early:	 9.5
tiempo late: 	 10.499999999999996
fecha:       	 None

------ NODO 5 -----
tiempo early:	 13.5
tiempo late: 	 14.499999999999996
fecha:       	 None

------ NODO 6 -----
tiempo early:	 11.5
tiempo late: 	 14.499999999999996
fecha:       	 None

------ NODO 7 -----
tiempo early:	 16.5
tiempo late: 	 17.499999999999996
fecha:       	 None

------ NODO 8 -----
tiempo early:	 14.5
tiempo late: 	 17.499999999999996
fecha:       	 None

------ NODO 9 -----
tiempo early:	 18.5
tiempo late: 	 21.499999999999996
fecha:       	 None

------ NODO 10 -----
tiempo early:	 7.5
tiempo late: 

In [112]:
grafoP.printCriticalPath()


--- ARISTA: 	 A1
--- ORIGEN: 	 0
--- DESTINO:	 1
--- Ti PERT:	 3.0
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A2
--- ORIGEN: 	 1
--- DESTINO:	 2
--- Ti PERT:	 3.0
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A10
--- ORIGEN: 	 2
--- DESTINO:	 10
--- Ti PERT:	 1.5
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A11
--- ORIGEN: 	 10
--- DESTINO:	 11
--- Ti PERT:	 2.0
--- HOLG  2:	 0.49999999999999645

--- ARISTA: 	 A13
--- ORIGEN: 	 10
--- DESTINO:	 13
--- Ti PERT:	 2.5
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A14
--- ORIGEN: 	 11
--- DESTINO:	 14
--- Ti PERT:	 4.0
--- HOLG  2:	 0.49999999999999645

--- ARISTA: 	 A15
--- ORIGEN: 	 11
--- DESTINO:	 15
--- Ti PERT:	 4.0
--- HOLG  2:	 0.49999999999999645

--- ARISTA: 	 A14
--- ORIGEN: 	 13
--- DESTINO:	 14
--- Ti PERT:	 4.0
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A15
--- ORIGEN: 	 13
--- DESTINO:	 15
--- Ti PERT:	 4.0
--- HOLG  2:	 -3.552713678800501e-15

--- ARISTA: 	 A16
--- ORIGEN: 	 14
--- DEST